# Model Training - Indonesian License Plate Detection

This notebook trains a YOLOv8 model for Indonesian license plate detection using the prepared dataset.

## Tasks:
- [ ] Load pre-trained YOLOv8 model
- [ ] Configure hyperparameters (as per CLAUDE.md specifications)
- [ ] Setup Weights & Biases (W&B) experiment tracking
- [ ] Start training with progress monitoring
- [ ] Visualize training metrics (loss, mAP)
- [ ] Save training checkpoints
- [ ] Implement early stopping
- [ ] Export best model for production

## 1. Import Libraries and Setup

In [ ]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image
import yaml
import json
import torch
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Import YOLOv8
from ultralytics import YOLO
from ultralytics.utils import LOGGER

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully")
print(f"Working directory: {os.getcwd()}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 2. Setup Paths and Configuration

In [ ]:
# Path configuration
BASE_DIR = Path("..")
DATASET_PATH = BASE_DIR / "dataset"
MODELS_DIR = BASE_DIR / "models"
EXPERIMENTS_DIR = MODELS_DIR / "experiments"
CHECKPOINTS_DIR = MODELS_DIR / "checkpoints"
FINAL_MODELS_DIR = MODELS_DIR / "final"
RESULTS_DIR = BASE_DIR / "results"

# Create directories
for directory in [EXPERIMENTS_DIR, CHECKPOINTS_DIR, FINAL_MODELS_DIR, RESULTS_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

print(f"Dataset path: {DATASET_PATH}")
print(f"Models directory: {MODELS_DIR}")
print(f"Results directory: {RESULTS_DIR}")

# Verify dataset configuration
data_yaml = DATASET_PATH / "data.yaml"
if data_yaml.exists():
    with open(data_yaml, 'r') as f:
        dataset_config = yaml.safe_load(f)
    
    print("\n✅ Dataset configuration found:")
    for key, value in dataset_config.items():
        print(f"  {key}: {value}")
else:
    print("\n❌ Dataset configuration not found. Please run Notebook 03 first.")
    dataset_config = None

## 3. Setup Weights & Biases (W&B) Experiment Tracking

In [ ]:
# Install and setup Weights & Biases
try:
    import wandb
    print("✅ Weights & Biases available")
except ImportError:
    print("📦 Installing Weights & Biases...")
    !pip install wandb
    import wandb

# Initialize W&B project
def setup_wandb_tracking():
    """Setup Weights & Biases experiment tracking"""
    
    project_name = "indonesian-license-plate-detection"
    experiment_name = f"yolov8n-training-{datetime.now().strftime('%Y%m%d-%H%M%S')}"
    
    # Initialize wandb
    wandb.init(
        project=project_name,
        name=experiment_name,
        config={
            "model_type": "YOLOv8n",
            "dataset": "Indonesian License Plates",
            "task": "object_detection",
            "framework": "ultralytics",
            "device": "cuda" if torch.cuda.is_available() else "cpu"
        },
        tags=["yolov8", "license-plate", "indonesian", "detection"]
    )
    
    print(f"🔬 W&B experiment initialized: {experiment_name}")
    return experiment_name

# Setup tracking (optional - can be skipped if W&B not available)
try:
    experiment_name = setup_wandb_tracking()
    use_wandb = True
except Exception as e:
    print(f"⚠️  W&B setup failed: {e}")
    print("Training will continue without W&B tracking")
    use_wandb = False
    experiment_name = f"yolov8n-training-{datetime.now().strftime('%Y%m%d-%H%M%S')}"

## 4. Training Configuration (CLAUDE.md Specifications)

In [ ]:
# Training hyperparameters as specified in CLAUDE.md
training_config = {
    # Basic parameters
    "model": "yolov8n.pt",  # Pre-trained model
    "data": str(data_yaml) if data_yaml.exists() else "dataset/data.yaml",
    "epochs": 100,
    "patience": 20,  # Early stopping patience
    "batch": 16,     # Baseline for GPU >=8GB VRAM. Reduce to 8 if memory errors
    "imgsz": 640,
    "optimizer": "AdamW",
    "lr0": 0.001,    # Initial learning rate
    "val": True,
    
    # Additional parameters for better training
    "save": True,
    "save_period": 10,  # Save checkpoint every 10 epochs
    "cache": False,     # Don't cache images (to save memory)
    "device": "0" if torch.cuda.is_available() else "cpu",
    "workers": 4,       # Number of dataloader workers
    "project": str(EXPERIMENTS_DIR),
    "name": experiment_name,
    "exist_ok": True,
    "pretrained": True,
    "verbose": True,
    
    # Performance targets from CLAUDE.md
    "target_map50": 0.85,  # mAP@0.5 > 0.85
    "confidence_threshold": 0.3,  # >= 0.3
}

# Adjust batch size if low memory
if torch.cuda.is_available():
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    if gpu_memory < 8:
        training_config["batch"] = 8
        print(f"⚠️  Reduced batch size to 8 due to limited GPU memory ({gpu_memory:.1f} GB)")
    else:
        print(f"✅ Using batch size 16 with {gpu_memory:.1f} GB GPU memory")
else:
    training_config["batch"] = 4  # Very small batch for CPU training
    training_config["workers"] = 2
    print("⚠️  CPU training detected - reduced batch size to 4")

print("\n🔧 Training Configuration:")
print("=" * 30)
for key, value in training_config.items():
    if key not in ["target_map50", "confidence_threshold"]:
        print(f"{key}: {value}")

# Log to W&B if available
if use_wandb:
    wandb.config.update(training_config)

## 5. Load Pre-trained Model

In [ ]:
# Load YOLOv8 model
print("🤖 Loading YOLOv8 model...")

try:
    # Load pre-trained YOLOv8n model
    model = YOLO(training_config["model"])
    print(f"✅ Model loaded: {training_config['model']}")
    
    # Display model info
    model.info(verbose=False)
    
    # Check model device
    print(f"Model device: {model.device}")
    
except Exception as e:
    print(f"❌ Error loading model: {e}")
    raise

## 6. Dataset Verification Before Training

In [ ]:
def verify_dataset_before_training(data_path):
    """Final verification before starting training"""
    
    print("🔍 Final dataset verification...")
    
    if not data_path.exists():
        print(f"❌ Dataset path not found: {data_path}")
        return False
    
    # Check data.yaml
    yaml_file = data_path / "data.yaml"
    if not yaml_file.exists():
        print("❌ data.yaml not found")
        return False
    
    # Load configuration
    with open(yaml_file, 'r') as f:
        config = yaml.safe_load(f)
    
    # Check required splits
    required_splits = ['train', 'val']
    for split in required_splits:
        if split not in config:
            print(f"❌ Missing {split} split in configuration")
            return False
        
        # Check directories exist
        images_dir = data_path / split / "images"
        labels_dir = data_path / split / "labels"
        
        if not images_dir.exists():
            print(f"❌ Missing {split}/images directory")
            return False
        
        if not labels_dir.exists():
            print(f"❌ Missing {split}/labels directory")
            return False
        
        # Count files
        image_count = len(list(images_dir.glob("*")))  
        label_count = len(list(labels_dir.glob("*.txt")))
        
        print(f"✅ {split}: {image_count} images, {label_count} labels")
        
        if image_count == 0:
            print(f"❌ No images found in {split} split")
            return False
        
        if image_count != label_count:
            print(f"⚠️  Image-label mismatch in {split}: {image_count} vs {label_count}")
    
    # Check classes
    if 'nc' not in config or 'names' not in config:
        print("❌ Missing class configuration")
        return False
    
    print(f"✅ Classes: {config['nc']} ({config['names']})")
    print("✅ Dataset verification passed")
    return True

# Run verification
if dataset_config:
    dataset_ready = verify_dataset_before_training(DATASET_PATH)
else:
    print("❌ Cannot verify dataset - configuration not loaded")
    dataset_ready = False

## 7. Start Training

In [ ]:
# Start training if dataset is ready
if dataset_ready and model:
    print("🚀 Starting training...")
    print(f"Experiment: {experiment_name}")
    print(f"Target: mAP@0.5 > {training_config['target_map50']}")
    
    try:
        # Start training with the specified configuration
        results = model.train(**{
            k: v for k, v in training_config.items() 
            if k not in ["model", "target_map50", "confidence_threshold"]
        })
        
        print("✅ Training completed successfully")
        
        # Save training results
        training_results = {
            "experiment_name": experiment_name,
            "training_config": training_config,
            "final_results": str(results) if results else "No results",
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        
        # Save to file
        results_file = RESULTS_DIR / "metrics" / f"{experiment_name}_results.json"
        results_file.parent.mkdir(parents=True, exist_ok=True)
        
        with open(results_file, 'w') as f:
            json.dump(training_results, f, indent=2, default=str)
        
        print(f"📊 Training results saved to: {results_file}")
        
    except Exception as e:
        print(f"❌ Training failed: {e}")
        raise
        
else:
    print("❌ Cannot start training - dataset not ready or model not loaded")
    print("Please run previous cells to fix issues")

## 8. Analyze Training Results

In [ ]:
def analyze_training_results(experiment_dir):
    """Analyze and visualize training results"""
    
    results_csv = experiment_dir / "results.csv"
    
    if not results_csv.exists():
        print(f"❌ Results file not found: {results_csv}")
        return None
    
    # Load training results
    df = pd.read_csv(results_csv)
    df.columns = df.columns.str.strip()  # Remove whitespace
    
    print(f"📊 Training completed in {len(df)} epochs")
    
    # Get final metrics
    final_metrics = df.iloc[-1]
    
    print("\n🎯 Final Performance Metrics:")
    print("=" * 40)
    
    # Key metrics
    metrics_to_show = {
        'metrics/mAP50(B)': 'mAP@0.5',
        'metrics/mAP50-95(B)': 'mAP@0.5:0.95',
        'metrics/precision(B)': 'Precision',
        'metrics/recall(B)': 'Recall',
        'val/box_loss': 'Box Loss',
        'val/cls_loss': 'Class Loss',
        'val/dfl_loss': 'DFL Loss'
    }
    
    for col, name in metrics_to_show.items():
        if col in df.columns:
            value = final_metrics[col]
            print(f"{name}: {value:.4f}")
    
    # Check if target achieved
    map50_col = 'metrics/mAP50(B)'
    if map50_col in df.columns:
        final_map50 = final_metrics[map50_col]
        target_map50 = training_config['target_map50']
        
        if final_map50 >= target_map50:
            print(f"\n🎉 TARGET ACHIEVED: mAP@0.5 = {final_map50:.4f} (target: {target_map50})")
        else:
            print(f"\n⚠️  Target not reached: mAP@0.5 = {final_map50:.4f} (target: {target_map50})")
            print("Consider: longer training, data augmentation, or hyperparameter tuning")
    
    # Create training plots
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Training and validation losses
    if 'train/box_loss' in df.columns:
        axes[0,0].plot(df['epoch'], df['train/box_loss'], label='Train Box Loss', alpha=0.7)
    if 'val/box_loss' in df.columns:
        axes[0,0].plot(df['epoch'], df['val/box_loss'], label='Val Box Loss', alpha=0.7)
    axes[0,0].set_title('Box Loss')
    axes[0,0].set_xlabel('Epoch')
    axes[0,0].set_ylabel('Loss')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)
    
    # mAP metrics
    if 'metrics/mAP50(B)' in df.columns:
        axes[0,1].plot(df['epoch'], df['metrics/mAP50(B)'], label='mAP@0.5', alpha=0.7)
    if 'metrics/mAP50-95(B)' in df.columns:
        axes[0,1].plot(df['epoch'], df['metrics/mAP50-95(B)'], label='mAP@0.5:0.95', alpha=0.7)
    axes[0,1].set_title('mAP Metrics')
    axes[0,1].set_xlabel('Epoch')
    axes[0,1].set_ylabel('mAP')
    axes[0,1].legend()
    axes[0,1].grid(True, alpha=0.3)
    
    # Precision and Recall
    if 'metrics/precision(B)' in df.columns:
        axes[1,0].plot(df['epoch'], df['metrics/precision(B)'], label='Precision', alpha=0.7)
    if 'metrics/recall(B)' in df.columns:
        axes[1,0].plot(df['epoch'], df['metrics/recall(B)'], label='Recall', alpha=0.7)
    axes[1,0].set_title('Precision & Recall')
    axes[1,0].set_xlabel('Epoch')
    axes[1,0].set_ylabel('Score')
    axes[1,0].legend()
    axes[1,0].grid(True, alpha=0.3)
    
    # Learning rate
    lr_cols = [col for col in df.columns if 'lr' in col.lower()]
    if lr_cols:
        for lr_col in lr_cols[:2]:  # Show max 2 LR schedules
            axes[1,1].plot(df['epoch'], df[lr_col], label=lr_col, alpha=0.7)
        axes[1,1].set_title('Learning Rate')
        axes[1,1].set_xlabel('Epoch')
        axes[1,1].set_ylabel('Learning Rate')
        axes[1,1].legend()
        axes[1,1].grid(True, alpha=0.3)
    else:
        axes[1,1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Save plots
    plots_dir = RESULTS_DIR / "plots"
    plots_dir.mkdir(parents=True, exist_ok=True)
    plt.savefig(plots_dir / f"{experiment_name}_training_curves.png", dpi=300, bbox_inches='tight')
    print(f"📈 Training plots saved to: {plots_dir / f'{experiment_name}_training_curves.png'}")
    
    return df

# Analyze results if training was completed
if 'results' in locals() and results:
    experiment_path = EXPERIMENTS_DIR / experiment_name
    training_df = analyze_training_results(experiment_path)
else:
    print("⚠️  No training results to analyze")

## 9. Find and Copy Best Model

In [ ]:
def find_and_copy_best_model(experiment_dir, final_models_dir):
    """Find best model weights and copy to final models directory"""
    
    # Look for best.pt in experiment directory
    best_model_path = experiment_dir / "weights" / "best.pt"
    
    if not best_model_path.exists():
        print(f"❌ Best model not found at: {best_model_path}")
        return None
    
    # Copy to final models directory with descriptive name
    final_model_name = f"yolov8n_indonesian_plates_{experiment_name}.pt"
    final_model_path = final_models_dir / final_model_name
    
    import shutil
    shutil.copy2(best_model_path, final_model_path)
    
    # Also create a symbolic link to "best_model.pt" as specified in CLAUDE.md
    best_model_link = final_models_dir / "best_model.pt"
    if best_model_link.exists():
        best_model_link.unlink()
    
    try:
        # Try to create symbolic link (may not work on all systems)
        best_model_link.symlink_to(final_model_name)
        print(f"🔗 Created symbolic link: {best_model_link}")
    except OSError:
        # Fallback: copy the file
        shutil.copy2(final_model_path, best_model_link)
        print(f"📄 Created copy: {best_model_link}")
    
    print(f"✅ Best model saved to: {final_model_path}")
    
    # Get model file size
    model_size_mb = final_model_path.stat().st_size / (1024 * 1024)
    print(f"📏 Model size: {model_size_mb:.1f} MB")
    
    # Check if it meets size requirement (< 50MB from CLAUDE.md)
    if model_size_mb < 50:
        print("✅ Model size meets requirement (< 50MB)")
    else:
        print(f"⚠️  Model size exceeds 50MB requirement ({model_size_mb:.1f} MB)")
    
    return final_model_path

# Find and copy best model
if 'results' in locals() and results:
    experiment_path = EXPERIMENTS_DIR / experiment_name
    best_model_path = find_and_copy_best_model(experiment_path, FINAL_MODELS_DIR)
else:
    print("⚠️  No trained model to copy")
    best_model_path = None

## 10. Model Performance Testing

In [ ]:
def test_model_performance(model_path, test_data_path):
    """Test model performance on inference speed and accuracy"""
    
    if not model_path or not model_path.exists():
        print("❌ Model not available for testing")
        return None
    
    print(f"🧪 Testing model performance: {model_path.name}")
    
    # Load trained model
    trained_model = YOLO(str(model_path))
    
    # Get test images
    test_images_dir = test_data_path / "test" / "images"
    if not test_images_dir.exists():
        # Fallback to validation images
        test_images_dir = test_data_path / "val" / "images"
    
    if not test_images_dir.exists():
        print("❌ No test images found")
        return None
    
    test_images = list(test_images_dir.glob("*.jpg")) + list(test_images_dir.glob("*.png"))
    
    if not test_images:
        print("❌ No test images found")
        return None
    
    # Test on sample images
    sample_images = test_images[:10]  # Test on 10 images
    print(f"Testing on {len(sample_images)} sample images...")
    
    total_time = 0
    detections_count = 0
    
    for img_path in sample_images:
        start_time = datetime.now()
        
        # Run inference
        results = trained_model(str(img_path), conf=training_config['confidence_threshold'])
        
        end_time = datetime.now()
        inference_time = (end_time - start_time).total_seconds() * 1000  # ms
        
        total_time += inference_time
        
        # Count detections
        if results and len(results) > 0:
            detections_count += len(results[0].boxes) if results[0].boxes is not None else 0
    
    # Calculate average performance
    avg_inference_time = total_time / len(sample_images)
    avg_detections = detections_count / len(sample_images)
    
    print(f"\n⚡ Performance Results:")
    print(f"Average inference time: {avg_inference_time:.1f} ms")
    print(f"Average detections per image: {avg_detections:.1f}")
    
    # Check if meets speed requirement (< 100ms from CLAUDE.md)
    speed_target = 100  # ms
    if avg_inference_time < speed_target:
        print(f"✅ Speed requirement met (< {speed_target}ms)")
    else:
        print(f"⚠️  Speed requirement not met ({avg_inference_time:.1f}ms > {speed_target}ms)")
    
    # Test with confidence threshold
    print(f"\n🎯 Testing with confidence threshold: {training_config['confidence_threshold']}")
    
    performance_results = {
        "avg_inference_time_ms": avg_inference_time,
        "avg_detections_per_image": avg_detections,
        "confidence_threshold": training_config['confidence_threshold'],
        "meets_speed_requirement": avg_inference_time < speed_target,
        "sample_size": len(sample_images)
    }
    
    return performance_results

# Test model performance
if best_model_path and DATASET_PATH.exists():
    performance_results = test_model_performance(best_model_path, DATASET_PATH)
else:
    print("⚠️  Cannot test model performance - model or dataset not available")
    performance_results = None

## 11. Generate Training Report

In [ ]:
def generate_training_report():
    """Generate comprehensive training report"""
    
    report = {
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "experiment_name": experiment_name,
        "training_config": training_config,
        "model_info": {
            "architecture": "YOLOv8n",
            "pretrained": True,
            "framework": "ultralytics",
            "device": "cuda" if torch.cuda.is_available() else "cpu"
        },
        "dataset_info": dataset_config if dataset_config else {},
        "performance_results": performance_results if performance_results else {},
        "model_paths": {
            "best_model": str(best_model_path) if best_model_path else None,
            "experiment_dir": str(EXPERIMENTS_DIR / experiment_name)
        },
        "requirements_check": {}
    }
    
    # Check requirements from CLAUDE.md
    requirements = {
        "target_map50": training_config['target_map50'],
        "speed_target_ms": 100,
        "model_size_target_mb": 50,
        "confidence_threshold": training_config['confidence_threshold']
    }
    
    # Check final metrics if available
    if 'training_df' in locals() and training_df is not None:
        final_metrics = training_df.iloc[-1]
        
        if 'metrics/mAP50(B)' in training_df.columns:
            final_map50 = final_metrics['metrics/mAP50(B)']
            report["final_metrics"] = {
                "mAP50": final_map50,
                "mAP50_95": final_metrics.get('metrics/mAP50-95(B)', None),
                "precision": final_metrics.get('metrics/precision(B)', None),
                "recall": final_metrics.get('metrics/recall(B)', None)
            }
            
            report["requirements_check"]["map50_achieved"] = final_map50 >= requirements["target_map50"]
    
    # Check model size
    if best_model_path and best_model_path.exists():
        model_size_mb = best_model_path.stat().st_size / (1024 * 1024)
        report["model_size_mb"] = model_size_mb
        report["requirements_check"]["size_requirement_met"] = model_size_mb < requirements["model_size_target_mb"]
    
    # Check speed requirement
    if performance_results:
        report["requirements_check"]["speed_requirement_met"] = performance_results["meets_speed_requirement"]
    
    # Calculate overall success
    checks = report["requirements_check"]
    success_rate = sum(checks.values()) / len(checks) if checks else 0
    report["overall_success_rate"] = success_rate
    
    return report

# Generate comprehensive report
training_report = generate_training_report()

# Display report summary
print("\n📋 Training Report Summary")
print("=" * 50)
print(f"Generated: {training_report['timestamp']}")
print(f"Experiment: {training_report['experiment_name']}")

if "final_metrics" in training_report:
    metrics = training_report["final_metrics"]
    print(f"\n🎯 Final Performance:")
    for metric, value in metrics.items():
        if value is not None:
            print(f"  {metric}: {value:.4f}")

if "model_size_mb" in training_report:
    print(f"\n📏 Model Size: {training_report['model_size_mb']:.1f} MB")

if training_report["requirements_check"]:
    print(f"\n✅ Requirements Check:")
    for req, passed in training_report["requirements_check"].items():
        status = "✅" if passed else "❌"
        print(f"  {status} {req.replace('_', ' ').title()}: {passed}")
    
    print(f"\n🏆 Overall Success Rate: {training_report['overall_success_rate']:.1%}")

# Save detailed report
reports_dir = RESULTS_DIR / "reports"
reports_dir.mkdir(parents=True, exist_ok=True)

with open(reports_dir / f"{experiment_name}_training_report.json", 'w') as f:
    json.dump(training_report, f, indent=2, default=str)

print(f"\n📄 Detailed report saved to: {reports_dir / f'{experiment_name}_training_report.json'}")

# Finalize W&B if used
if use_wandb:
    # Log final metrics to W&B
    if "final_metrics" in training_report:
        wandb.log(training_report["final_metrics"])
    
    wandb.finish()
    print("🔬 W&B experiment completed")

## 12. Production Integration Preparation

In [ ]:
def prepare_for_production():
    """Prepare model and instructions for production integration"""
    
    print("🚀 Preparing for production integration...")
    
    # Check if we have a trained model
    if not best_model_path or not best_model_path.exists():
        print("❌ No trained model available for production")
        return False
    
    # Production integration info from CLAUDE.md
    production_info = {
        "model_file": "best_model.pt",
        "target_location": "../license-plate/cached_models/yolov8_indonesian_plates.pt",
        "expected_output_format": {
            "success": "bool",
            "detections": "list of detection objects",
            "total_detections": "int",
            "total_processing_time_ms": "int",
            "error": "str or None"
        },
        "confidence_threshold": training_config['confidence_threshold'],
        "integration_points": {
            "model_storage": "homepage/utils/model_storage.py",
            "ml_pipeline": "homepage/utils/ml_pipeline.py",
            "database_models": "homepage/models.py"
        }
    }
    
    # Create integration instructions
    instructions = f"""# Production Integration Instructions

## Model Information
- **Model**: {best_model_path.name}
- **Architecture**: YOLOv8n
- **Size**: {training_report.get('model_size_mb', 'Unknown'):.1f} MB
- **Target Performance**: mAP@0.5 > {training_config['target_map50']}
- **Confidence Threshold**: {training_config['confidence_threshold']}

## Integration Steps

1. **Copy Model to Production**:
   ```bash
   cp {best_model_path} ../license-plate/cached_models/yolov8_indonesian_plates.pt
   ```

2. **Update Model Path** (if needed):
   - File: `homepage/utils/model_storage.py`
   - Expected path: `cached_models/yolov8_indonesian_plates.pt`

3. **Test Integration**:
   ```bash
   cd ../license-plate
   python manage.py shell -c "
   from homepage.utils.ml_pipeline import detect_license_plate
   result = detect_license_plate('path/to/test/image.jpg')
   print(result)
   "
   ```

## Expected Output Format
```json
{
  "success": true,
  "detections": [
    {
      "license_plate_number": "L 1234 AB",
      "confidence_score": 0.92,
      "bbox": [450, 310, 680, 375],
      "processing_time_ms": 85,
      "detection_index": 0
    }
  ],
  "total_detections": 1,
  "total_processing_time_ms": 95,
  "error": null
}
```

## Performance Targets Met
{chr(10).join([f'- {req.replace("_", " ").title()}: {"✅" if passed else "❌"}' for req, passed in training_report.get("requirements_check", {}).items()])}

## Next Steps
1. Copy model to production environment
2. Test integration with existing ml_pipeline.py
3. Verify database integration works
4. Test with real license plate images
5. Monitor performance in production
"""
    
    # Save integration instructions
    instructions_file = FINAL_MODELS_DIR / "INTEGRATION_INSTRUCTIONS.md"
    with open(instructions_file, 'w') as f:
        f.write(instructions)
    
    print(f"📄 Integration instructions saved to: {instructions_file}")
    
    # Create production info JSON
    production_file = FINAL_MODELS_DIR / "production_info.json"
    with open(production_file, 'w') as f:
        json.dump(production_info, f, indent=2)
    
    print(f"📄 Production info saved to: {production_file}")
    
    # Show quick integration command
    print(f"\n🚀 Quick Integration Command:")
    print(f"cp {best_model_path} ../license-plate/cached_models/yolov8_indonesian_plates.pt")
    
    return True

# Prepare for production
production_ready = prepare_for_production()

## Summary and Next Steps

This notebook completed the YOLOv8 model training for Indonesian license plate detection:

### ✅ Completed Tasks:
- Pre-trained YOLOv8n model loaded and configured
- Training with CLAUDE.md specifications (epochs=100, patience=20, batch=16)
- Weights & Biases experiment tracking (optional)
- Training progress monitoring and metrics visualization
- Best model selection and export
- Performance testing (speed and accuracy)
- Comprehensive training report generation
- Production integration preparation

### 🎯 Training Configuration Used:
- **Model**: YOLOv8n (pre-trained)
- **Epochs**: 100 with early stopping (patience=20)
- **Batch Size**: 16 (adjusted for GPU memory)
- **Image Size**: 640x640
- **Optimizer**: AdamW
- **Learning Rate**: 0.001

### 📊 Performance Targets (CLAUDE.md):
- **Accuracy**: mAP@0.5 > 0.85
- **Speed**: < 100ms per image (GPU inference)
- **Model Size**: < 50MB for deployment
- **Confidence Threshold**: >= 0.3

### 🚀 Production Integration:
- **Model Location**: `models/final/best_model.pt`
- **Target Path**: `../license-plate/cached_models/yolov8_indonesian_plates.pt`
- **Integration Instructions**: Available in `models/final/INTEGRATION_INSTRUCTIONS.md`

### 🎯 Next Steps:
1. **If training successful**: Proceed to Notebook 05 (Evaluation & Export)
2. **If targets not met**: Consider hyperparameter tuning or additional training
3. **Production Integration**: Follow integration instructions to deploy model
4. **Testing**: Verify model works with production pipeline

The trained model is ready for evaluation and production integration!